In [45]:
import requests
import pandas as pd
import time

In [52]:
#input video url
referer = input("Enter video url: ")

#test url
#referer = 'https://www.tiktok.com/@toanchodien23/video/7121230509027478810'

#get video id from referer - example: 7121230509027478810
id = referer.replace('?','/').split('/')[5]
print(id)


In [47]:
#Create DataFrame to store comments
df = pd.DataFrame(columns=['cid', 'text', 'time', 'digg_count', 'reply_comment_total', 'user_id', 'user_nickname', 'user_unique_id', 'referer'])

#Load dataframe from csv file if you have data set already
#df = pd.read_csv('comments.csv')

#---------------------------------
#cid: comment id
#text: comment
#time: comment time
#digg_count: number of likes
#reply_comment_total: number of replies
#user_id: user id
#user_nickname: user nickname
#user_unique_id: user unique id
#referer: video url

In [48]:
#Epoch & Unix Timestamp Conversion to GMT+0 time
#Cuz Tiktok API return Epoch & Unix Timestamp, we need to convert it to GMT+0 time
def epoch_to_date(epoch):
    return time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(epoch))

#If you want to convert it to local time, use this function
# def epoch_to_date(epoch):
#     return time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(epoch))

In [49]:
headers = {
    'authority': 'www.tiktok.com',
    'accept': '*/*',
    'accept-language': 'en,vi;q=0.9',    
    'referer': referer,
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36 Edg/105.0.1343.27',
}

#set cursor - address of the first row
cursor = 0

#set count - number of rows to read - maximum 50 rows per request
count = 50

In [ ]:
#Cuz TikTok API only return {count} comments per request, so we need to loop to get all comments
f = 0
while True:
    f += 1
    try:
        response = requests.get(f'https://www.tiktok.com/api/comment/list/?aweme_id={id}&count={count}&cursor={cursor}', headers=headers)
        json_data = response.json()
        cursor = json_data['cursor']
        for data in json_data['comments']:
            #append data to dataframe
            df = df.append({'cid':data['cid'], 'text':data['text'], 'time':epoch_to_date(data['create_time']), 'digg_count':data['digg_count'], 'reply_comment_total':data['reply_comment_total'], 'user_id':data['user']['uid'], 'user_nickname':data['user']['nickname'], 'user_unique_id':data['user']['unique_id'], 'referer':referer}, ignore_index=True)
            f += 1
    except:
        print(f"Error at {f}th request")
        break

In [54]:
#Dataframe description
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   cid                  71 non-null     object
 1   text                 71 non-null     object
 2   time                 71 non-null     object
 3   digg_count           71 non-null     object
 4   reply_comment_total  71 non-null     object
 5   user_id              71 non-null     object
 6   user_nickname        71 non-null     object
 7   user_unique_id       71 non-null     object
 8   referer              71 non-null     object
dtypes: object(9)
memory usage: 5.1+ KB


,cid,text,time,digg_count,reply_comment_total,user_id,user_nickname,user_unique_id,referer
0,7135706856186872602,khớp thì vẫn mòn mà a,2022-08-25 07:07:17,21,11,6854993900370150402,concaocao,pl4yz0n3,https://www.tiktok.com/@daoyeumoc/video/713566...
1,7135670325429158682,Có màu trắng ko ạ,2022-08-25 04:45:22,4,3,7099765229395919873,Không Tên,khongten_535,https://www.tiktok.com/@daoyeumoc/video/713566...
2,7135669054211293978,Gian nhà ngày xưa dùng kiểu khớp như này đúng ...,2022-08-25 04:40:29,2,1,6863452083518161921,NorthSide,babylearnmath12,https://www.tiktok.com/@daoyeumoc/video/713566...
3,7135668433228628737,lâu quá mới thấy lên vidoe lạiii,2022-08-25 04:39:37,2,0,6946053496815780865,bunnie._.bunnie,bunnie._.bunnie,https://www.tiktok.com/@daoyeumoc/video/713566...
4,7135686046478779163,cái ghế vòm hôm bữa đâu rồi anh?,2022-08-25 05:46:31,1,1,7073487819280909313,Chuyên,c.hyue,https://www.tiktok.com/@daoyeumoc/video/713566...


In [55]:
#save to csv
df.to_csv('TikTok_comments_1.csv', index=False)